# Constellation and Chain Analysis: Transmitters and Receivers Example

<img src="TransmitterAndReceiver.jpg" alt="Drawing" style="width: 800px;"/>

**Terminology**
* Node = Object in STK
* Edge = Access between two objects in STK
* Strand = The sequence of nodes and edges to complete access in a chain

**This notebook shows how to:**
* Find the least latency path between a starting constellation and ending constellation through a network of receivers and transmitters. The connecting constellations in the scenario can have logical constraints so they can only transmit to receivers on other satellites
* Typical STK constraints such as range, link duration, Eb/No, etc can be taken into account
*  Compute summary statistics for time delay and number of hops
* Load the shortest path data back into STK



In [22]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 70)
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.utilities.colors import Colors
from agi.stk12.stkobjects import *
from agi.stk12.stkutil import *
from agi.stk12.vgt import *
import seaborn as sns
import matplotlib.pyplot as plt
from chainPathLib2 import *
import time
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

## Constellation Names, Computation Time, and Processing Delays

In [23]:
# Inputs for Chain Analysis
startingConstellation = 'Targets' # Will be used as the start of the chain
firstConnectingConstellation = 'ObservingSatsReceivers' # Will be used to connect the the starting and ending constellation
secondConnectingConstellation = 'ObservingSatsTransmitters' # Will be used to connect the the starting and ending constellation
endingConstellation = 'EndLocations' # Will be used as the end of the chain

start = 0 # EpSec
stop = 3600*1 # EpSec
metric = 'timeDelay' # 'distance','timeDelay' # could optionally add: min hops+secondary metric, fewest hand off etc
nodeDelays = {'ObservingSatsReceivers':0.005,'ObservingSatsTransmitters':0.005} # Add in time delays. Provide the constellation name in STK and the node delays
stkVersion = 12

In [24]:
# Connect to STK
stkApp = STKDesktop.AttachToApplication()
stkRoot = stkApp.Root
stkRoot.UnitPreferences.SetCurrentUnit('DateFormat','EpSec')
stkRoot.ExecuteCommand('Units_SetConnect / Date "EpochSeconds"')
stkRoot.ExecuteCommand('VO * ObjectLine DeleteAll')

In [25]:
# # Example of how to adjust all transmitters power
# transmitterPaths = FilterObjectsByType(stkRoot,'Transmitter')
# for transmitterPath in transmitterPaths:
#     transmitter = stkRoot.GetObjectFromPath(transmitterPath)
#     transmitter2 = transmitter.QueryInterface(STKObjects.IAgTransmitter)
#     transmitterModel = transmitter2.Model.QueryInterface(STKObjects.IAgTransmitterModelSimple)
#     transmitterModel.Eirp = 16.9

In [27]:
# Build chains and add node processing delays
if not secondConnectingConstellation:
    chainNames = createChains(stkRoot,startingConstellation,firstConnectingConstellation,endingConstellation)
else:
    chainNames = createChains(stkRoot,startingConstellation,firstConnectingConstellation,endingConstellation,secondConnectingConstellation=secondConnectingConstellation)
print(chainNames)
nodeDelaysByNode = getNodeDelaysByNode(stkRoot,nodeDelays,chainNames=chainNames)


['StartingToConnecting', 'FirstToSecond', 'SecondToFirst', 'ConnectingToEnding']


STKRuntimeError: Invalid object path.

## Compute  Strands and Distances

In [ ]:
# Compute strands
t1 = time.time()
strands,dfStrands = getAllStrands(stkRoot,chainNames,start,stop)
print(time.time()-t1)
dfStrands

In [ ]:
# Compute node positions, distances and time delays

# Time resolution of distance calculation
step = 10 # sec

t1 = time.time()
nodesTimesPos = computeNodesPosOverTime(stkRoot,strands,start,stop,step) # Pull node position over time
t2 = time.time()
print(t2-t1)

t1 = time.time()
strandsAtTimes = getStrandsAtTimes(strands,start,stop,step) # Discretize strand intervals into times
t2 = time.time()
print(t2-t1)

t1 = time.time()
timeNodePos = computeTimeNodePos(strandsAtTimes,nodesTimesPos) # Nodes and positions at each time
t2 = time.time()
print(t2-t1)

t1 = time.time()
timesEdgesDistancesDelays = computeTimeEdgesDistancesDelays(strandsAtTimes,nodesTimesPos,nodeDelaysByNode)  # Edges, distances and delays at each time
t2 = time.time()
print(t2-t1)

## Use NX for Network Metrics and Reliability Analysis


In [ ]:
# Get starting nodes and ending nodes
startingNodes = getNodesFromConstellation(stkRoot,startingConstellation)
endingNodes = getNodesFromConstellation(stkRoot,endingConstellation)
startingNodes,endingNodes

In [ ]:
# Build new networks at each time and gather metrics
t1 = time.time()

strandsShort = []
distances = []
timeStrandMetric = []
setsOfMinNodesToRemoveAll = []
setsOfMinNodesToRemoveAny = []
i = 0
for t in np.arange(start,stop+step,step):
    # Generate Network at each time
    if secondConnectingConstellation:
        G = generateDiNetwork(t,timesEdgesDistancesDelays,timeNodePos) # A directed network is needed when using two connecting constellations
    else:
        G = generateNetwork(t,timesEdgesDistancesDelays,timeNodePos)

    # Find shortest strand distance
    if any([node in G.nodes() for node in startingNodes]) and any([node in G.nodes() for node in endingNodes]):
        strandShort,metricVal = shortestStrandDistance(G,startingNodes,endingNodes,metric=metric)
        timeStrandMetric.append((t,strandShort,metricVal))
    else:
        timeStrandMetric.append((t,'',np.nan))

df = pd.DataFrame(timeStrandMetric,columns=['time','strand',metric])
df['num hops'] = df['strand'].apply(lambda x: len(x)-2)
df.loc[df['num hops'] < 0,'num hops'] = np.nan
df[metric] = df[metric].astype(float)
if setsOfMinNodesToRemoveAny:
    numSets = [len(sets) for sets in setsOfMinNodesToRemoveAll]
    lengthOfSet = [len(sets[0]) if len(sets) > 0 else 0 for sets in setsOfMinNodesToRemoveAll]
    df['min nodes to lose access to all'] = lengthOfSet
    df['num of sets to lose access to all'] = numSets
if setsOfMinNodesToRemoveAny:
    numSets = [len(sets) for sets in setsOfMinNodesToRemoveAny]
    lengthOfSet = [len(sets[0]) if len(sets) > 0 else 0 for sets in setsOfMinNodesToRemoveAny]
    df['min nodes to lose access to any'] = lengthOfSet
    df['num of sets to lose access to any'] = numSets
print(time.time()-t1)
df

In [ ]:
# Plot metric over time
df.plot.line(x='time',y=metric);

In [ ]:
# Plot metric over time
df.plot.line(x='time',y='num hops')

In [ ]:
# Create intervals for the strands and add to STK
dfIntervals = createDfIntervals(df,stop,step)
addStrandsAsObjectLines(stkRoot,dfIntervals,color='yellow')

In [ ]:
# Add data back into STK
t1 = time.time()
addDataToSTK(stkRoot,chainNames[0],df)
print(time.time()-t1)

## Example Scenario and Statistics

In [ ]:
# Most active nodes
strands = dfIntervals[['strand','start','stop']].values
dfNodesIntervals = getNodesIntervalsFromStrands(strands)
dfNodeActive = getActiveDuration(dfNodesIntervals,start,stop)
dfNodeActive.sort_values('sum dur',ascending=False).head(10)

In [ ]:
# Summary stats for dur for each interval gap
dfIntervals[dfIntervals['strand'] == ''].describe()['dur']

In [ ]:
# Look at a strand
t = 0
list(df.loc[df['time'] == t]['strand'])

## Add constraint

In [ ]:
# # Add a constraint and then recompute
# receiverPaths = FilterObjectsByType(stkRoot,'Receiver')
# for receiverPath in receiverPaths:
#     receiver = stkRoot.GetObjectFromPath(receiverPath)
#     try:
#         cnConstraint = receiver.AccessConstraints.AddNamedConstraint('C/N')
#     except:
#         cnConstraint = receiver.AccessConstraints.GetActiveNamedConstraint('C/N')
#     cnConstraint = cnConstraint.QueryInterface(STKObjects.IAgAccessCnstrMinMax)
#     cnConstraint.EnableMin = True
#     cnConstraint.Min = 1